In [55]:
import pandas as pd
from typing import Tuple, List, Dict
from tabulate import tabulate
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import string
import nltk
import pandas as pd
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [56]:
notspam = np.arange(0, 193)
spam = np.arange(0, 18)

df_list = []

for ns in notspam : 
    df_tmp1 = pd.read_csv('BaiThi2/TrainData/notspam/{}_notspam.txt'.format(ns))
    df_tmp1.drop(df_tmp1.columns[[0]], axis = 1, inplace = True)
    f = open('BaiThi2/TrainData/notspam/{}_notspam.txt'.format(ns), "r")
    df_tmp1['message'] = f.read()
    df_tmp1['Label'] = 'notspam'
    df_list.append(df_tmp1)

for s in spam : 
    df_tmp2 = pd.read_csv('BaiThi2/TrainData/spam/{}_spam.txt'.format(s))
    df_tmp2.drop(df_tmp2.columns[[0]], axis = 1, inplace = True)
    f2 = open('BaiThi2/TrainData/spam/{}_spam.txt'.format(s), "r")
    df_tmp2['message'] = f2.read()
    df_tmp2['Label'] = 'spam'
    df_list.append(df_tmp2)


In [57]:
df_list[0]

,,,,,,,,,message,Label
> date : sun,"15 dec 91 02 : 25 : 02 est > : michael < mmorse @ vm1 . yorku . ca > > subject : re : 2 . 864 queries > > wlodek zadrozny asks is "" anything interesting "" said > construction "" s > np np "" . . . second","> much related : might consider construction form > has been discussed list late reduplication ? > logical sense "" john mcnamara name "" is tautologous thus",> level,"indistinguishable """,,"here ? "" . ' john mcnamara name ' is tautologous is support those logic-based semantics is irrelevant natural language . sense is tautologous ? supplies value attribute followed attribute is value . fact value name-attribute relevant entity were ' chaim shmendrik '",' john mcnamara name ' false . tautology,. ( reduplication,Subject: re : 2 . 882 s - > np np\n\n> date : ...,notspam


In [58]:
# df.drop(df.columns[[0]], axis = 1, inplace = True)
df = pd.concat(df_list).reset_index(drop = True)
# df.drop(df.columns[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]], axis = 1, inplace = True)
df = df.iloc[ : , 0 : 2]
df

,message,Label
0,Subject: re : 2 . 882 s - > np np\n\n> date : ...,notspam
1,Subject: s - > np + np\n\ndiscussion s - > np ...,notspam
2,Subject: 2 . 882 s - > np np\n\n. . . 's much ...,notspam
3,"Subject: gent conference\n\n"" listserv "" inter...",notspam
4,Subject: query : causatives korean\n\nanyone p...,notspam
...,...,...
206,Subject: `\n\nhello ! our naughty little city ...,spam
207,Subject: attract women ! ! ( 30740 )\n\nattrac...,spam
208,Subject: are serious . . . help retire 2 3 yea...,spam
209,"Subject: "" complete home business kit ! ! ""\n\...",spam


In [59]:
def text_preprocess(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    nopunc = nopunc.lower()
    
    # Now just remove any stopwords and non alphabets
    nostop=[word for word in nopunc.split() if word.lower() not in stopwords.words('english') and word.isalpha()]
    
    return nostop

In [60]:
spam_messages = df[df["Label"] == "spam"]["message"]
notspam_messages = df[df["Label"] == "notspam"]["message"]
print("No of spam messages : ",len(spam_messages))
print("No of notspam messages : ",len(notspam_messages))

No of spam messages :  18
No of notspam messages :  193


In [61]:
# Download stopwords
nltk.download('stopwords')

# Words in spam messages
spam_words = []
for each_message in spam_messages:
    spam_words += text_preprocess(each_message)
    
print(f"Top 10 spam words are:\n {pd.Series(spam_words).value_counts().head(10)}")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Top 10 spam words are:
 report         77
email          49
money          42
information    38
offshore       38
mail           33
order          29
subject        27
sell           25
business       25
dtype: int64


In [62]:
df["message"] = df["message"].apply(text_preprocess)
df.head()


,message,Label
0,"[subject, np, np, date, sun, dec, est, michael...",notspam
1,"[subject, np, np, discussion, np, np, reminds,...",notspam
2,"[subject, np, np, much, restrictive, np, np, n...",notspam
3,"[subject, gent, conference, listserv, internat...",notspam
4,"[subject, query, causatives, korean, anyone, p...",notspam


In [63]:
# Convert messages (as lists of string tokens) to strings
df["message"] = df["message"].agg(lambda x: " ".join(map(str, x)))
df.head()


,message,Label
0,subject np np date sun dec est michael mmorse ...,notspam
1,subject np np discussion np np reminds years a...,notspam
2,subject np np much restrictive np np np pro qu...,notspam
3,subject gent conference listserv international...,notspam
4,subject query causatives korean anyone point b...,notspam


In [64]:
vectorizer = CountVectorizer()
bow_transformer = vectorizer.fit(df["message"])

# Fetch the vocabulary set
print(f"20 BOW Features: {vectorizer.get_feature_names()[20:40]}")
print(f"Total number of vocab words: {len(vectorizer.vocabulary_)}")

20 BOW Features: ['ableist', 'ablock', 'aborigen', 'aborigenes', 'aboriginal', 'abound', 'abrahamsen', 'abreviaturas', 'abril', 'absence', 'absolute', 'absolutely', 'abstract', 'abstractness', 'abstracts', 'abstractsdeadline', 'ac', 'academe', 'academia', 'academic']
Total number of vocab words: 10343


c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [65]:
# Convert strings to vectors using BoW
messages_bow = bow_transformer.transform(df["message"])

# Print the shape of the sparse matrix and count the number of non-zero occurrences
print(f"Shape of sparse matrix: {messages_bow.shape}")
print(f"Amount of non-zero occurrences: {messages_bow.nnz}")

Shape of sparse matrix: (211, 10343)
Amount of non-zero occurrences: 26221


In [66]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(messages_bow)

# Transform entire BoW into tf-idf corpus
messages_tfidf = tfidf_transformer.transform(messages_bow)
print(messages_tfidf.shape)

(211, 10343)


In [67]:
# Convert spam and ham labels to 0 and 1 (or, vice-versa)
FactorResult = pd.factorize(df["Label"])
df["Label"] = FactorResult[0]
df.head()

,message,Label
0,subject np np date sun dec est michael mmorse ...,0
1,subject np np discussion np np reminds years a...,0
2,subject np np much restrictive np np np pro qu...,0
3,subject gent conference listserv international...,0
4,subject query causatives korean anyone point b...,0


In [69]:
# Split the dataset to train and test sets
msg_train, msg_test, label_train, label_test = train_test_split(
    messages_tfidf, df["Label"], test_size=0.2
)
print(msg_train)
print(f"train dataset features size: {msg_train.shape}")
print(f"train dataset label size: {label_train.shape}")

print(f"test dataset features size: {msg_test.shape}")
print(f"test dataset label size: {label_test.shape}")

  (0, 9820)	0.1369406638798837
  (0, 9560)	0.1758358780611115
  (0, 9378)	0.1758358780611115
  (0, 9287)	0.351671756122223
  (0, 9244)	0.12020614596536597
  (0, 8929)	0.031047544634834577
  (0, 8499)	0.14172666399231057
  (0, 8182)	0.1758358780611115
  (0, 8181)	0.16324718201925525
  (0, 8077)	0.15431536003416688
  (0, 7673)	0.10593879545783934
  (0, 7467)	0.16324718201925525
  (0, 7352)	0.09502875388165342
  (0, 7331)	0.1758358780611115
  (0, 7283)	0.15431536003416688
  (0, 6221)	0.11127432398027758
  (0, 5709)	0.1758358780611115
  (0, 5674)	0.1758358780611115
  (0, 5352)	0.1758358780611115
  (0, 5309)	0.1758358780611115
  (0, 4245)	0.16324718201925525
  (0, 3842)	0.1758358780611115
  (0, 3538)	0.08524626102251089
  (0, 2970)	0.15431536003416688
  (0, 2862)	0.07182958138903842
  :	:
  (167, 2504)	0.06774509912578264
  (167, 2433)	0.11762882252746124
  (167, 2342)	0.056828293965888864
  (167, 2289)	0.23380150945070907
  (167, 2174)	0.05881441126373062
  (167, 2028)	0.07296921663072864


In [ ]:
# Train an xgboost classifier
from xgboost import XGBClassifier

# Instantiate our model
clf = XGBClassifier()

# Fit the model to the training data
clf.fit(msg_train, label_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [74]:
# Make predictions
predict_train = clf.predict(msg_test)

print(
    f"Accuracy of Train dataset: {metrics.accuracy_score(label_test, predict_train):0.3f}"
)

Accuracy of Train dataset: 1.000


In [ ]:
no_labels = np.arange(0, 78)
df_noLabel = []

for nl in no_labels : 
    df_unknown = pd.read_csv('BaiThi2/TestData_nolabel/{}_unknown.txt'.format(nl))
    df_unknown.drop(df_unknown.columns[[0]], axis = 1, inplace = True)
    f = open('BaiThi2/TestData_nolabel/{}_unknown.txt'.format(nl), "r")
    df_unknown['message'] = f.read()
    df_noLabel.append(df_unknown)
    

In [ ]:
df_noLabel1 = pd.concat(df_noLabel).reset_index(drop = True)
df_noLabel1 = df_noLabel1.iloc[ : , 0 : 1]
df_noLabel1

,message
0,Subject: base generated adjuncts\n\ndoes anyon...
1,Subject: basic journals\n\nare facing major cu...
2,Subject: query : tagalog philippine informants...
3,Subject: ancient vocal tract simulation\n\nwor...
4,"Subject: re : 3 . 386 chomsky , mac concordanc..."
...,...
73,Subject: really cool hot videos !\n\nattention...
74,"Subject: improvements wound care !\n\nhello , ..."
75,Subject: \n\nb r g n r f r e s 1 - stop travel...
76,"Subject: credit program "" guaranteed credit ""\..."


In [ ]:
df_noLabel1["message"] = df_noLabel1["message"].apply(text_preprocess)
df_noLabel1.head()

,message
0,"[subject, base, generated, adjuncts, anyone, r..."
1,"[subject, basic, journals, facing, major, cuts..."
2,"[subject, query, tagalog, philippine, informan..."
3,"[subject, ancient, vocal, tract, simulation, w..."
4,"[subject, chomsky, mac, concordance, seems, wh..."


In [ ]:
# Convert messages (as lists of string tokens) to strings
df_noLabel1["message"] = df_noLabel1["message"].agg(lambda x: " ".join(map(str, x)))
df_noLabel1.head()
df_noLabel1
# clf.predict(df_noLabel1)


,message
0,subject base generated adjuncts anyone referen...
1,subject basic journals facing major cuts libra...
2,subject query tagalog philippine informants na...
3,subject ancient vocal tract simulation work fo...
4,subject chomsky mac concordance seems whatever...
...,...
73,subject really cool hot videos attention warni...
74,subject improvements wound care hello name kev...
75,subject b r g n r f r e stop travel supplier a...
76,subject credit program guaranteed credit bad c...


In [ ]:
# vectorizer1 = CountVectorizer()
# # bow_transformer = vectorizer1.fit(df_noLabel1["message"])

# # Fetch the vocabulary set
# print(f"20 BOW Features: {vectorizer1.get_feature_names()[20:40]}")
# print(f"Total number of vocab words: {len(vectorizer1.vocabulary_)}")

In [75]:
# Convert strings to vectors using BoW
messages_bow_NL = bow_transformer.transform(df_noLabel1["message"])

# Print the shape of the sparse matrix and count the number of non-zero occurrences
print(f"Shape of sparse matrix: {messages_bow_NL.shape}")
print(f"Amount of non-zero occurrences: {messages_bow_NL.nnz}")

Shape of sparse matrix: (78, 10343)
Amount of non-zero occurrences: 7822


In [77]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(messages_bow_NL)

# Transform entire BoW into tf-idf corpus
messages_tfidf_NL = tfidf_transformer.transform(messages_bow_NL)
print(messages_tfidf_NL.shape)

(78, 10343)


In [80]:
from sklearn.metrics import classification_report, f1_score, ConfusionMatrixDisplay, confusion_matrix

In [87]:
model_pred = clf.predict(messages_tfidf_NL)
model_pred = pd.DataFrame(model_pred)
model_pred = model_pred.replace({0: 'nospam', 1: 'spam'})
print(model_pred)
model_pred = model_pred.to_csv('ANSWER.csv')


         0
0   nospam
1   nospam
2   nospam
3   nospam
4   nospam
..     ...
73    spam
74    spam
75  nospam
76  nospam
77  nospam

[78 rows x 1 columns]
